# import

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
import seaborn as sns
import xgboost as xgb
import time

%matplotlib inline

# preprocess (train_test)

## train_test (feature outlier)

In [49]:
# load data
train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])

train_id = train.id
test_id = test.id
train_columns = train.columns.tolist()
test_columns = test.columns.tolist()


def tridx(col):
    if col not in train_columns:
        return -1
    return train_columns.index(col)

def tsidx(col):
    if col not in test_columns:
        return -1
    return test_columns.index(col)

# 纠正个别错误
# @life_sq
equal_index = [601, 1896, 2791]
test.iloc[equal_index, tsidx('life_sq')] = test.iloc[equal_index, tsidx('full_sq')]

# @build_year
kitch_is_build_year = [13117]
train.iloc[kitch_is_build_year, tridx('build_year')] = train.iloc[kitch_is_build_year, tridx('kitch_sq')]

# @state
train.loc[train.state == 33, 'state'] = np.NaN

# process bad index, fill with nan
train_test = pd.concat([train, test])
is_train = train_test.id.isin(train_id.unique())
is_test = train_test.id.isin(test_id.unique())

# 标记离群点
# @life_sq
isbad_life_sq = ((train_test.life_sq > train_test.full_sq) | 
                 (train_test.life_sq < 5) |
                 ((train_test.life_sq > 300) & is_train) |
                 ((train_test.life_sq > 200) & is_test))
isbad_life_sq_id = train_test.loc[isbad_life_sq, 'id']
train_test.loc[isbad_life_sq, 'life_sq'] = np.NaN
print 'bad_life_sq', np.sum(isbad_life_sq)

# @full_sq
isbad_full_sq = ((train_test.full_sq < 5) |
                 ((train_test.full_sq > 210) & (train_test.life_sq / train_test.full_sq < 0.3) & is_train) |
                 ((train_test.full_sq > 150) & (train_test.life_sq / train_test.full_sq < 0.3) & is_test) |
                 ((train_test.life_sq > 300) & is_train) |
                 ((train_test.life_sq > 200) & is_test))
isbad_full_sq_id = train_test.loc[isbad_full_sq, 'id']
train_test.loc[isbad_full_sq, 'full_sq'] = np.NaN
print 'bad_full_sq', np.sum(isbad_full_sq)

# @kitch_sq
isbad_kitch_sq = ((train_test.id == 13120) |
                  (train_test.kitch_sq > train_test.life_sq) |
                  (train_test.kitch_sq == 0) |
                  (train_test.kitch_sq == 1))
isbad_kitch_sq_id = train_test.loc[isbad_kitch_sq, 'id']
train_test.loc[isbad_kitch_sq, 'kitch_sq'] = np.NaN
print 'bad_kitch_sq', np.sum(isbad_kitch_sq)

# @build_year
isbad_build_year = ((train_test.build_year < 1500) |
                    (train_test.build_year > 2200))
isbad_build_year_id = train_test.loc[isbad_build_year, 'id']
train_test.loc[isbad_build_year, 'build_year'] = np.NaN
print 'bad_build_year', np.sum(isbad_build_year)

# @num_room
isbad_num_room_selected_id = train_test.iloc[[10076, 11621, 17764, 19390, 24007, 26713, 29172, 3174, 7313]].id.unique()
isbad_num_room = ((train_test.id.isin(isbad_num_room_selected_id)) |
                  (train_test.num_room == 0))
isbad_num_room_id = train_test.loc[isbad_num_room, 'id']
train_test.loc[isbad_num_room, 'num_room'] = np.NaN
print 'bad_num_room', np.sum(isbad_num_room)

# @floor
isbad_floor = ((train_test.floor > train_test.max_floor)|
               (train_test.floor == 0))
isbad_floor_id = train_test.loc[isbad_floor, 'id']
train_test.loc[isbad_floor, 'num_room'] = np.NaN
print 'bad_floor', np.sum(isbad_floor)

# @max_floor
isbad_max_floor = ((train_test.floor > train_test.max_floor)|
                   (train_test.max_floor == 0))
isbad_max_floor_id = train_test.loc[isbad_max_floor, 'id']
train_test.loc[isbad_max_floor, 'num_room'] = np.NaN
print 'max_floor', np.sum(isbad_max_floor)

bad_life_sq 815
bad_full_sq 36
bad_kitch_sq 8417
bad_build_year 1463
bad_num_room 23
bad_floor 2145
max_floor 2140


## train_clean & train_ex (target outlier)

In [51]:


train = train_test.loc[train_test.id.isin(train_id.unique())]
test = train_test.loc[train_test.id.isin(test_id.unique())]

train_outlier = ((train.price_doc/train.full_sq > 600000) |
                 (train.price_doc/train.full_sq < 10000))
train_outlier_id = train.loc[train_outlier].id
print 'train_outlier', np.sum(train_outlier)

train_clean = train.loc[~train_outlier].copy()
train_ex = train.loc[train_outlier].copy()

train_outlier 41


## train_test additional features

In [ ]:
# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek

# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)

test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)


# preprocess (macro)

# single xgb